In [1]:
### SBiDer


import SBiDer_helper as helper
import sqlite3
import node

# sbider_input_analyzer

In [2]:
def grammar_0(cursor, tokens):
    """
    Grammar for 'grammar_0:= grammar_1 = grammar_1'.
    """
    
    if len(tokens) < 3 or '=' not in tokens:
        raise ValueError("grammar_0: Usage: input = output")
        
    else:
        input_output_dictionary = helper.split_by(tokens, '=')

    return grammar_output(grammar_1(cursor, input_output_dictionary[0]), 
                          grammar_1(cursor, input_output_dictionary[1]))


def grammar_1(cursor, tokens):
    """
    Grammar for 'grammar_1:= grammar_2 or grammar_1 | grammar_2 and grammar_1 | grammar_2'.
    """
    
    if len(tokens) > 1 and tokens[1] == 'or':
        # grammar_2 or grammar_1
        # split tokens by the first occurring 'or' and store the tokens before
        # and after the 'or' in a dictionary
        or_dictionary = helper.split_by(tokens, 'or')
        return grammar_or(grammar_2(cursor, or_dictionary.get(0)), grammar_1(cursor, or_dictionary.get(1)))

    elif len(tokens) > 1 and tokens[1] == 'and':
        # grammar_2 and grammar_1
        # split tokens by the first occurring 'and' and store the tokens
        # before and after the 'and' in a dictionary
        and_dictionary = helper.split_by(tokens, 'and')
        return grammar_and(grammar_2(cursor, and_dictionary.get(0)), grammar_1(cursor, and_dictionary.get(1)))

    else:
        # grammar_2
        return grammar_2(cursor, tokens)


def grammar_2(cursor, tokens):
    """
    Grammar for 'grammar_2:= (grammar_1) or grammar_1 | (grammar_1) and grammar_1 | (grammar_1) | interactor'.
    """

    if len(tokens) <= 0:
        raise ValueError("Invalid Syntax")

    elif tokens[0] == "(":
        # (grammar_1) or grammar_1 | (grammar_1) and grammar_1| (grammar_1)

        # token after the last occurring ')'
        token_after_last_closer = helper.remove_parentheses(tokens)

        if token_after_last_closer == 'or':
            # split tokens by the first occurring 'or' and store the tokens
            # before and after the 'or' in a dictionary

            or_dictionary = helper.split_by(tokens, 'or')
            return grammar_or(grammar_1(cursor, or_dictionary.get(0)), grammar_1(cursor, or_dictionary.get(1)))

        elif token_after_last_closer == 'and':
            # split tokens by the first occurring 'and' and store the tokens
            # before and after the 'and' in a dictionary

            and_dictionary = helper.split_by(tokens, 'and')
            return grammar_and(grammar_1(and_dictionary.get(0)), grammar_1(and_dictionary.get(1)))

        else:
            # grammar_1; delegate to grammar_1
            return grammar_1(cursor, tokens)

    else:
        # interactor; delegate to interactor
        return interactor(cursor, tokens)


def interactor(cursor, token):
    """
    Grammar for 'interactor'.
    """

    species = token[0]
    return [[db_get_species_id_from_name(cursor, species)]]


def grammar_output(tokens1, tokens2):
    """
    Grammar for '='.
    """

    grammar_output_dict = {}

    for token1 in tokens1:
        grammar_output_dict[tuple(token1)] = tuple(tokens2)

    return grammar_output_dict


def grammar_or(tokens1, tokens2):
    """
    Grammar for 'or'.
    """

    return tokens1 + tokens2


def grammar_and(tokens1, tokens2):
    """
    Grammar for 'and'.
    """

    grammar_and_output = []

    for token1 in tokens1:
        for token2 in tokens2:
            grammar_and_output.append(helper.uniquely_merge_list_of_lists([token1, token2]))

    return grammar_and_output


def parse_logic(cursor, logic_input):
    """
    Parse a logic input into one or more atomized and equivalent logics.
    """

    split_logic_input = logic_input.split()

    # begins recursive logic parse
    return grammar_0(cursor, split_logic_input)

# sbider_database_manager

### SQL command maker

In [3]:
def make_sql_insert_command(table_name, table_header_list, insert_data_list):
    """
    Make SQL insert command.
    """
    
    sql_insert_into = 'INSERT INTO ' + table_name
    
    headers = '(' + ','.join(table_header_list) + ')'

    insert_data_list_formatted = list(insert_data_list)
    for index in range(len(insert_data_list_formatted)):
        if isinstance(insert_data_list_formatted[index], str):
            insert_data_list_formatted[index] = "'" + insert_data_list_formatted[index] + "'"
        else:
            insert_data_list_formatted[index] = str(insert_data_list_formatted[index])
    values = 'Values (' + ','.join(insert_data_list_formatted) + ')'

    return sql_insert_into + '\n\t' + headers + '\n\t' + values + ';'


def make_sql_select_command(table_name, table_header_list, where_columns=None, where_options=None,
                            where_values=None, where_bools=None, group=None, having_columns=None, having_bools=None,
                            having_values=None):
    """
    Make SQL select command.
    """

    if where_columns is not None and where_options is not None and where_values is not None and where_bools is not None:
        if (len(where_columns) != len(where_options) and len(where_options) != len(where_values) and len(
                where_values) != (len(where_bools) - 1)):
            raise Exception("Invalid argument")
    elif where_columns is not None or where_options is not None or where_values is not None or where_bools is not None:
        raise Exception("Invalid argument")

    # must have a table name
    if table_name is None or len(table_name) == 0:
        raise Exception("a table name must be provided.")

    sql_select_command = "SELECT "
    if table_header_list == "*":
        sql_select_command += " * "
    else:
        for table_header_index in range(len(table_header_list)):
            sql_select_command += table_header_list[table_header_index]
            if table_header_index != len(table_header_list) - 1:
                sql_select_command += ", "
            else:
                sql_select_command += " "
    sql_select_command += "\n" + "FROM " + table_name + " "

    if where_columns is not None:
        sql_select_command += "\n" + "WHERE "
        for where_index in range(len(where_columns)):
            sql_select_command += where_columns[where_index] + " " + where_options[where_index] + " " + str(
                where_values[where_index]) + " "
            if where_index < len(where_bools):
                sql_select_command += where_bools[where_index] + " "

    if group is not None:
        sql_select_command += "\n" + "GROUP BY " + group

    if having_columns is not None and having_bools is not None and having_values is not None:
        sql_select_command += "\n" + "HAVING " + having_columns + " " + having_bools + " " + str(having_values)
    sql_select_command += ";"

    return sql_select_command


def make_sql_update_command(table_name, table_header_list, update_data_list, where_column="",
                            where_option="", where_value=""):
    """
    Makes SQL update command.
    """

    sql_update = 'UPDATE ' + table_name

    update_values_list = []
    for column_name, update_value in zip(table_header_list, update_data_list):
        update_values_list.append(str(column_name) + ' = ' + str(update_value))

    sql_update_values = 'SET ' + ', '.join(update_values_list)

    sql_where = ""
    if where_column != "":
        sql_where = "\n" + " ".join(['WHERE', where_column, where_option, where_value])

    '''for where_column_name, where_value , where_op, index in 
            zip(where_columns, where_values, where_options, range(len(where_columns) + 1)):
            if i < len(where_columns) - 1:
                hold = sql_where + ' '.join([where_column_name,where_option,str(where_value)]) + ' ' + w_conts[index]
            else:
                hold = sql_where + ' '.join([where_column_name,where_option,str(where_value)]) + ' ' 
            sql_where = hold 
        return update_command + '\n\t' + set_str + '\n\t' + where_str + ';'''

    sql_update_command = sql_update + "\n" + sql_update_values + sql_where + ";"

    return sql_update_command


def make_sql_delete_command(table_name):
    sql_delete_command = "DELETE FROM %s;" % table_name
    return sql_delete_command

### SBiDer database builder

In [4]:
def db_open(database_file):
    """
    Connect to a database, or create a database if it does not exist.
    """

    connection = sqlite3.connect(database_file)
    connection.text_factory = str
    cursor = connection.cursor()
    return connection, cursor


def db_close(connection, cursor):
    """
    Close a database.
    """

    connection.commit()
    cursor.close()


def db_create_table(cursor):
    """
    Make database tables.
    """

    #db_drop_all_table(cursor)

    ##SQL commands
    
    # species id - species name - species type
    species = '''CREATE TABLE Species (spe_id VARCHAR(50), 
                                       name VARCHAR(50), 
                                       type VARCHAR(50));'''
    
    # plasmid id - plasmid name - pubmed id
    plasmid = '''CREATE TABLE Plasmid (pla_id VARCHAR(50), 
                                       name VARCHAR(50), 
                                       PMID VARCHAR(50));'''
    # operon id - operon name - image path
    operon = '''CREATE TABLE Operon (ope_id VARCHAR(50), 
                                     name VARCHAR(50),
                                     image VARCHAR(50));'''

    # operon id - plasmid id - direction
    po = '''CREATE TABLE PlasmidOperon (ope_id VARCHAR(50), 
                                        pla_id VARCHAR(50),
                                        direction VARCHAR(50));'''

    # input transition id - operon id
    oit = '''CREATE TABLE OperonInputTransition (it_id VARCHAR(50), 
                                                 ope_id VARCHAR(50));'''

    # input transition id - logic
    it = '''CREATE TABLE InputTransition (it_id VARCHAR(50), 
                                          logic VARCHAR(50));'''

    # input id - input transition id - species id - reverse
    in_ = '''CREATE TABLE InputTransitionSpecies (in_id VARCHAR(50), 
                                                  it_id VARCHAR(50), 
                                                  spe_id VARCHAR(50),
                                                  reverse BOOL);'''

    # output transition id - operon id
    oot = '''CREATE TABLE OperonOutputTransition (ot_id VARCHAR(50),
                                                  ope_id VARCHAR(50));'''

    # output transition id - logic
    ot = '''CREATE TABLE OutputTransition (ot_id VARCHAR(50), 
                                           logic VARCHAR(50))'''

    # output id - output id - species id
    out = '''CREATE TABLE OutputTransitionSpecies (out_id VARCHAR(50), 
                                                   ot_id VARCHAR(50),
                                                   spe_id VARCHAR(50));'''

    # user id - first name - last name - email - password
    login = '''CREATE TABLE User (user_id VARCHAR(50), 
                                  first_name VARCHAR(50),
                                  last_name VARCHAR(50),
                                  email VARCHAR(50),
                                  password VARCHAR(50));'''

    sql_make_table_commands = [species,
                               plasmid,
                               operon,
                               po,
                               oit,
                               it,
                               in_,
                               oot,
                               ot,
                               out,
                               login]

    for sql_make_table_command in sql_make_table_commands:
        cursor.execute(sql_make_table_command)
    return cursor


def db_drop_table(cursor, table_name):
    """
    Drop a database table.
    """
        
    cursor.execute("DROP TABLE %s;" % table_name)
    return cursor


def db_drop_all_table(cursor):
    """
    Drop all database tables.
    """

    table_names = ["Species",
                   "Plasmid",
                   "Operon",
                   "PlasmidOperon",
                   "OperonInputTransition",
                   "InputTransition",
                   "InputTransitionSpecies",
                   "OperonOutputTransition",
                   "OutputTransition",
                   "OutputTransitionSpecies",
                   "User"]
    for table_name in table_names:
        cursor.execute("DROP TABLE %s;" % table_name)
    return cursor

### Database accessor

In [5]:
def db_print_table(cursor, table_name):
    """
    Print a table.
    """

    cursor.execute("SELECT * FROM " + table_name)
    table = cursor.fetchall()
    print(table_name + ' table:')
    for l in table:
        print(l)
    print('\n')


def db_print_all_tables(cursor):
    """
    Print all database tables.
    """
    db_print_table(cursor, "Species")
    db_print_table(cursor, "Plasmid")
    db_print_table(cursor, "Operon")
    db_print_table(cursor, "PlasmidOperon")
    db_print_table(cursor, "OperonInputTransition")
    db_print_table(cursor, "InputTransition")
    db_print_table(cursor, "InputTransitionSpecies")
    db_print_table(cursor, "OperonOutputTransition")    
    db_print_table(cursor, "OutputTransition")
    db_print_table(cursor, "OutputTransitionSpecies")
    db_print_table(cursor, "User")

    
def db_insert(cursor, table_name, table_header_list, insert_data_list):
    """
    Insert into a SQL table.
    """

    sql_command = make_sql_insert_command(table_name, table_header_list, insert_data_list)
    cursor.execute(sql_command)
    return cursor


def db_select(cursor, table_name, table_header_list, where_columns=None, where_options=None,
                  where_values=None, where_bools=None, group=None, having_columns=None, having_bools=None,
                  having_values=None):
    """
    Select from a SQL table.
    """

    sql_command = make_sql_select_command(table_name, table_header_list, where_columns, where_options,
                                          where_values, where_bools, group, having_columns, having_bools, having_values)
    cursor.execute(sql_command)
    return cursor


def db_update(cursor, table_name, table_header_list, update_data_list,
              where_column="", where_option="", where_value=""):
    """
    Update a SQL table.
    """

    sql_command = make_sql_update_command(table_name, table_header_list, update_data_list,
                                          where_column, where_option, where_value)
    cursor.execute(sql_command)
    return cursor


def db_delete(cursor, table_name):
    """
    Delete a SQL table.
    """
    cursor.execute(make_sql_delete_command(table_name))
    
    
def db_get_species_id_from_name(cursor, species_name):
    """
    Get species id from species name.
    """
    
    a_cur = db_select(cursor,
                      "Species",
                      ["spe_id"],
                      ["name"],
                      ["="],
                      ["'%s'" % species_name.lower()],
                      "")
    return a_cur.fetchone()[0]


def db_get_species_name_from_id(cursor, species_id):
    """
    Get species name from species id.
    """
    
    a_cur = db_select(cursor,
                      "Species",
                      ["name"],
                      ["spe_id"],
                      ["="],
                      [species_id],
                      "")
    return a_cur.fetchone()[0]


def db_get_operon_id_from_name(cursor, operon_name):
    """
    Get operon id from operon name.
    """
    
    a_cur = db_select(cursor,
                      "Operon",
                      ["ope_id"],
                      ["name"],
                      ["="],
                      ["'%s'" % operon_name],
                      "")
    return a_cur.fetchone()[0]


def db_get_operon_name_from_id(cursor, operon_id):
    """
    Get operon name from operon id.
    """
    
    a_cur = db_select(cursor,
                      "Operon",
                      ["name"],
                      ["ope_id"],
                      ["="],
                      ["'%s'" % operon_id],
                      "")
    return a_cur.fetchone()[0]


def db_get_plasmid_id_from_name(cursor, plasmid_name):
    """
    Get plasmid id from plasmid name.
    """
    
    a_cur = db_select(cursor,
                      "Plasmid",
                      ["pla_id"],
                      ["name"],
                      ["="],
                      ["'%s'" % plasmid_name.lower()],
                      "")
    return a_cur.fetchone()[0]


def db_get_plasmid_name_from_id(cursor, plasmid_id):
    """
    Get plasmid name from plasmid id.
    """
    
    a_cur = db_select(cursor,
                      "Plasmid",
                      ["name"],
                      ["pla_id"],
                      ["="],
                      [plasmid_id],
                      "")
    return a_cur.fetchone()[0]


def get_all_input_transition_species(cursor, input_transition_id):
    """
    Get all species an input transition requires for activation.
    """

    species_list = []
    species_list_unformatted = db_select(cursor,
                                         "InputTransitionSpecies",
                                         ["spe_id"],
                                         ["it_id"],
                                         ["="],
                                         ["'" + input_transition_id + "'"],
                                         [""])
    species_list_unformatted = species_list_unformatted.fetchall()
    
    for species_index in range(len(species_list_unformatted)):
        species_list.append(list(species_list_unformatted[species_index]))
        
    species_list = helper.uniquely_merge_multi_dimensional_list_of_lists(species_list)
    return species_list


def get_all_output_transition_species(cursor, input_transition_id):
    """
    Get all species an output transition produces.
    """
    
    species_list = []
    species_list_unformatted = db_select(cursor,
                                         "OutputTransitionSpecies",
                                         ["spe_id"],
                                         ["ot_id"],
                                         ["="],
                                         ["'" + input_transition_id + "'"],
                                         [""])
    species_list_unformatted = species_list_unformatted.fetchall()

    for species_index in range(len(species_list_unformatted)):
        species_list.append(list(species_list_unformatted[species_index]))

    species_list = helper.uniquely_merge_multi_dimensional_list_of_lists(species_list)
    return species_list

### Dictionary maker

In [6]:
def make_input_ope_id_spe_id_dic(cursor):
    """
    Make operon input species dictionary.
    """

    input_ope_id_spe_id_dict = {}

    merged_ope_it_spe = cursor.execute('''SELECT OperonInputTransition.ope_id, OperonInputTransition.it_id, InputTransitionSpecies.spe_id 
                                          FROM OperonInputTransition, InputTransitionSpecies 
                                          WHERE OperonInputTransition.it_id = InputTransitionSpecies.it_id''')

    previous_operon, previous_input_transition, previous_species = merged_ope_it_spe.fetchone()
    
    input_transition_list_idx = 0
    
    input_ope_id_spe_id_dict[previous_operon] = [[]]
    input_ope_id_spe_id_dict[previous_operon][input_transition_list_idx].append(previous_species.strip())

    for operon, input_transition, species in merged_ope_it_spe.fetchall():
        
        if operon == previous_operon:
            if input_transition == previous_input_transition:
                input_ope_id_spe_id_dict[operon][input_transition_list_idx].append(species.strip())
            elif input_transition != previous_input_transition:
                input_transition_list_idx += 1
                input_ope_id_spe_id_dict[operon].append([])
                input_ope_id_spe_id_dict[operon][input_transition_list_idx].append(species.strip())
                previous_input_transition = input_transition
        else:
            input_transition_list_idx = 0
            input_ope_id_spe_id_dict[operon] = [[]]
            input_ope_id_spe_id_dict[operon][input_transition_list_idx].append(species.strip())
            previous_operon = operon
            previous_input_transition = input_transition

    return input_ope_id_spe_id_dict


def make_output_ope_id_spe_id_dic(cursor):
    """
    Make operon output species dictionary.
    """
    
    output_ope_id_spe_id_dict = {}

    merged_ope_ot_spe = cursor.execute('''SELECT OperonOutputTransition.ope_id, OperonOutputTransition.ot_id, OutputTransitionSpecies.spe_id 
                                          FROM OperonOutputTransition, OutputTransitionSpecies 
                                          WHERE OperonOutputTransition.ot_id = OutputTransitionSpecies.ot_id''')

    # previous ope_id, ot_id, and spe_id
    previous_operon, previous_output_transition, previous_species = merged_ope_ot_spe.fetchone()

    output_transition_list_idx = 0

    output_ope_id_spe_id_dict[previous_operon] = [[]]
    output_ope_id_spe_id_dict[previous_operon][output_transition_list_idx].append(previous_species.strip())

    # ope_id, ot_id, and spe_id
    for operon, output_transition, species in merged_ope_ot_spe.fetchall():

        if operon == previous_operon and not helper.contain_all_elements(output_ope_id_spe_id_dict[operon], [species]):
            if output_transition == previous_output_transition:
                output_ope_id_spe_id_dict[operon][output_transition_list_idx].append(species.strip())
            else:
                output_transition_list_idx += 1
                output_ope_id_spe_id_dict[operon].append([])
                output_ope_id_spe_id_dict[operon][output_transition_list_idx].append(species.strip())
        else:
            output_transition_list_idx = 0
            output_ope_id_spe_id_dict[operon] = [[]]
            output_ope_id_spe_id_dict[operon][output_transition_list_idx].append(species.strip())
            previous_operon = operon
            previous_output_transition = output_transition

    return output_ope_id_spe_id_dict


def make_ope_id_spe_id_dics(cursor):
    """
    Make operon input species and operon output species dictionaries.
    """
    
    return make_input_ope_id_spe_id_dic(cursor), make_output_ope_id_spe_id_dic(cursor)


def make_ope_id_rep_spe_id_dic(cursor):
    """
    Make operon input-not dictionary.
    """

    input_ope_id_spe_not_dict = {}
    merged_ope_it_spe_not = cursor.execute('''SELECT OperonInputTransition.ope_id, OperonInputTransition.it_id, InputTransitionSpecies.spe_id, InputTransitionSpecies.repression
                                              FROM OperonInputTransition, InputTransitionSpecies 
                                              WHERE OperonInputTransition.it_id = InputTransitionSpecies.it_id''')

    previous_operon, previous_input_transition, previous_species, not_bool = merged_ope_it_spe_not.fetchone()

    input_transition_list_idx = 0

    input_ope_id_spe_not_dict[previous_operon] = [[]]

    if not_bool == "TRUE":
        input_ope_id_spe_not_dict[previous_operon][input_transition_list_idx].append(previous_species.strip())

    for operon, input_transition, species, not_bool in merged_ope_it_spe_not.fetchall():
        if operon == previous_operon:
            
            if not_bool == "TRUE":
                if input_transition == previous_input_transition:
                    input_ope_id_spe_not_dict[operon][input_transition_list_idx].append(species.strip())
                elif input_transition != previous_input_transition:
                    input_transition_list_idx += 1
                    input_ope_id_spe_not_dict[operon].append([])
                    input_ope_id_spe_not_dict[operon][input_transition_list_idx].append(species.strip())
                    previous_input_transition = input_transition

        else:
            input_transition_list_idx = 0
            input_ope_id_spe_not_dict[operon] = [[]]
            if not_bool == "TRUE":
                input_ope_id_spe_not_dict[operon][input_transition_list_idx].append(species.strip())
            previous_operon = operon
            previous_input_transition = input_transition

    return input_ope_id_spe_not_dict


def make_plasmid_species_name_dictionary(cursor, operon_id_plasmid_name_dictionary, operon_species_dictionary):
    plasmid_species_name_dictionary = {}
    for operon_id, species_id_list in operon_species_dictionary.items():
        uniquely_merge_spe_id_list = helper.uniquely_merge_multi_dimensional_list_of_lists(species_id_list)
        plasmid_name = operon_id_plasmid_name_dictionary[operon_id]
        plasmid_species_name_dictionary[plasmid_name] = [db_get_species_name_from_id(cursor, spe_id) for spe_id in
                                                         uniquely_merge_spe_id_list]
    return plasmid_species_name_dictionary


def make_pla_name_spe_name_dics(cursor):
    """Make operon input and output species dictionary."""

    #plasmid_name_input_species_name_dictionary = {}
    #plasmid_name_output_species_name_dictionary = {}

    operon_id_plasmid_name_dictionary = {}
    #species_id_to_name_dictionary = {}

    input_operon_species_dictionary, output_operon_species_dictionary = make_ope_id_spe_id_dics(cursor)

    # make operon_id plasmid_name dictionary
    merged_ope_id_pla_name = cursor.execute('''SELECT PlasmidOperon.ope_id,
                                                   Plasmid.name
                                            FROM PlasmidOperon,
                                                 Plasmid
                                            WHERE PlasmidOperon.pla_id = Plasmid.pla_id''')
    for ope_id, pla_name in merged_ope_id_pla_name.fetchall():
        operon_id_plasmid_name_dictionary[ope_id] = pla_name

    input_plasmid_species_name_dictionary = make_plasmid_species_name_dictionary(cursor,
                                                                                 operon_id_plasmid_name_dictionary,
                                                                                 input_operon_species_dictionary)
    output_plasmid_species_name_dictionary = make_plasmid_species_name_dictionary(cursor,
                                                                                  operon_id_plasmid_name_dictionary,
                                                                                  output_operon_species_dictionary)
    return input_plasmid_species_name_dictionary, output_plasmid_species_name_dictionary


def operon_PMC_dictionary(database):
    conn, cur = db_open(database)
    operon_PMC_dict = {}
    operon_PMC = cur.execute('''SELECT PlasmidOperon.ope_id, Plasmid.PMID
                                FROM PlasmidOperon, Plasmid
                                WHERE PlasmidOperon.pla_id == Plasmid.pla_id''')
    
    for ope_id, PMC_ID in operon_PMC:
        operon_PMC_dict[ope_id] = PMC_ID.replace(' PMID: ', '')
    db_close(conn, cur)
    return operon_PMC_dict

# SBiDer_searcher

In [7]:
def search_sbider_path_memory(input_dictionary, activated_paths, from_operon):
    activated_ope_dic = {}
    activated_spe_dic = {}
    for path_idx, ope_spe_path in enumerate(activated_paths):
        activated_ope_dic[path_idx] = ope_spe_path[0]
        activated_spe_dic[path_idx] = ope_spe_path[1]
    final_operon_requirement = input_dictionary[from_operon]
    activating_ope_list = []
    for path_idx, spe_produced in activated_spe_dic.items():
        for a_spe_produced in spe_produced:
            for and_spe_required in final_operon_requirement:
                if a_spe_produced in and_spe_required:
                    activating_ope_list.extend(activated_ope_dic.get(path_idx))
    return activating_ope_list


def build_sbider_path_memory_tree(input_dictionary, activated_paths, start_operon):
    root_ope = node.Node(start_operon)
    temp_queue_ope = [root_ope]
    temp_memory = []
    while len(activated_paths) > 0 and len(temp_queue_ope) > 0:
        from_node = temp_queue_ope.pop(0)
        from_operon = from_node.value
        children_operon = search_sbider_path_memory(input_dictionary, activated_paths, from_operon)
        if len(children_operon) > 0:
            for child_operon in children_operon:
                if child_operon not in temp_memory:
                    child_node = node.Node(child_operon)
                    from_node.append_child(child_node)
                    temp_queue_ope.append(child_node)
                    temp_memory.append(child_operon)
    return root_ope.get_path_from_all_leaf()


def build_indirect_sbider_path(input_dictionary,
                               repressor_dictionary,
                               output_dictionary,
                               input_species_list,
                               output_species_list,
                               path_queue,
                               final_operon_path_list,
                               memory_operon,
                               memory_species,
                               activated_paths):
    temp_memory_species = []
    for an_operon in set(input_dictionary.keys()) - set(memory_operon):

        if helper.promoter_activation(input_dictionary, repressor_dictionary, an_operon, [], memory_species, True):

            just_produced_species = output_dictionary[an_operon]
            just_produced_unique_species = helper.uniquely_merge_multi_dimensional_list_of_lists(just_produced_species)

            if helper.match_any_list(just_produced_species, output_species_list):

                if len(activated_paths) > 1:
                    ope_path_backward = build_sbider_path_memory_tree(input_dictionary,
                                                                      activated_paths,
                                                                      an_operon)
                    final_operon_path_list.extend(ope_path_backward)
            else:
                if an_operon not in memory_operon:
                    path_queue.append(([an_operon], just_produced_unique_species))
                    memory_operon.append(an_operon)
                    memory_operon = helper.remove_duplicates_within_list(memory_operon)
                    temp_memory_species.extend(just_produced_unique_species)
                    activated_paths.append([[an_operon], just_produced_unique_species])

    memory_species.extend(temp_memory_species)
    memory_species = helper.remove_duplicates_within_list(memory_species)

    if len(path_queue) > 0:
        build_direct_sbider_path(input_dictionary,
                                 repressor_dictionary,
                                 output_dictionary,
                                 input_species_list,
                                 output_species_list,
                                 path_queue,
                                 final_operon_path_list,
                                 memory_operon,
                                 memory_species,
                                 activated_paths,
                                 True)


def build_direct_sbider_path(input_dictionary,
                             repressor_dictionary,
                             output_dictionary,
                             input_species_list,
                             output_species_list,
                             path_queue,
                             final_operon_path_list,
                             memory_operon,
                             memory_species,
                             activated_paths,
                             indirect_flag):
    while len(path_queue) != 0:

        (previously_visited_operon_list, just_previously_produced_species_list) = path_queue.pop(0)

        for an_operon in set(input_dictionary.keys()) - set(
                helper.uniquely_merge_multi_dimensional_list_of_lists(previously_visited_operon_list)):
            if an_operon not in memory_operon:


                if helper.promoter_activation(input_dictionary, repressor_dictionary, an_operon,
                                              just_previously_produced_species_list, memory_species, False):

                    visited_operon_list = previously_visited_operon_list + [an_operon]
                    just_produced_species = output_dictionary[an_operon]
                    just_produced_unique_species = helper.uniquely_merge_multi_dimensional_list_of_lists(
                        just_produced_species)

                    if helper.match_any_list(just_produced_species, output_species_list):

                        if not indirect_flag:
                            final_operon_path_list.append(visited_operon_list)
                    else:
                        path_queue.append((visited_operon_list, just_produced_unique_species))
                        memory_operon.append(an_operon)
                        memory_operon = helper.remove_duplicates_within_list(memory_operon)
                        memory_species.extend(just_produced_unique_species)
                        memory_species = helper.remove_duplicates_within_list(memory_species)

                    activated_paths.append([[an_operon], just_produced_unique_species])

    if indirect_flag:
        build_indirect_sbider_path(input_dictionary,
                                   repressor_dictionary,
                                   output_dictionary,
                                   input_species_list,
                                   output_species_list,
                                   path_queue,
                                   final_operon_path_list,
                                   memory_operon,
                                   memory_species,
                                   activated_paths)
    return final_operon_path_list


def get_sbider_path(inp_dic,
                    rep_dic,
                    outp_dic,
                    inp_spe,
                    outp_spe,
                    indirect_flag=False):
    final_ope_path = []
    path_queue = [([], inp_spe)]
    memory_ope = []
    memory_spe = []
    memory_spe.extend(inp_spe)
    activated_paths = []
    build_direct_sbider_path(inp_dic,
                             rep_dic,
                             outp_dic,
                             inp_spe,
                             outp_spe,
                             path_queue,
                             final_ope_path,
                             memory_ope,
                             memory_spe,
                             activated_paths,
                             indirect_flag)
    if len(final_ope_path) > 0:
        final_ope_path = helper.remove_duplicated_lists_within_a_list_of_lists(final_ope_path)

    return final_ope_path

In [8]:
def build_sbider_network(dir_database, user_query, indirect=False):

    # Access database
    f_database = dir_database + "/SBiDer.db"
    conn, cur = db_open(f_database)
    
    # Print all tables
    #db_print_all_tables(cur)

    # Dictionary of fragmented user inputs that satisfy user query
    logic_dictionary = parse_logic(cur, user_query)

    # Dictionaries of: Operon <-> InputSpecies & Operon <-> OutputSpecies
    input_dictionary, output_dictionary = make_ope_id_spe_id_dics(cur)

    # Dictionary of: Operon <-> Repressor
    repressor_dictionary = make_ope_id_rep_spe_id_dic(cur)

    # Build operon path for each fragmented user input, which satisfies user query
    all_operon_path = []
    for input_species, output_species_list in logic_dictionary.items():

        operon_path_per_start_species = [input_species]
        for output_species in output_species_list:
            operon_path_list = get_sbider_path(input_dictionary,
                                                        repressor_dictionary,
                                                        output_dictionary,
                                                        list(input_species),
                                                        output_species,
                                                        indirect)

            operon_path_per_start_species.extend(operon_path_list)
        all_operon_path.append(operon_path_per_start_species)
        
        return operon_path_per_start_species

        # Create JSON file needed to display the found genetic circuit
        #path_json = grapher.create_subnetwork_json_string(cur, operon_path_per_start_species, database_file)

        #return path_json

"""
##Arguments:

# 0=sbider_network_builder.py
# 1=database path 
# 2=user input
# 3=indirect flag
"""
if __name__ == "__main__":

    # path to the directory that contains the SBiDer database
    dir_database = sys.argv[1]

    # boolean flag for indirect path
    last_argv = str(sys.argv[-1]).lower()

    if last_argv == 't':
        user_input = " ".join(sys.argv[2:-1:])
        indirect_flag = True

    elif last_argv == 'f':
        user_input = " ".join(sys.argv[2:-1:])
        indirect_flag = False

    else:
        user_input = " ".join(sys.argv[2::])
        indirect_flag = False

    print("*dir_database " + dir_database)
    print("*user input " + user_input)
    print("*indirect flag "+ str(indirect_flag) +"\n")

    # search SBiDer network
    final_path_json = build_sbider_network(dir_database, user_input, indirect_flag)
    
    # print out the final JSON path
    print(final_path_json)

#SBiDer tester

In [15]:
def test(inpt):

    # Input
    user_input = inpt
    
    # Accesss database
    dir_database = '/Users/Kwat/BInf/SBiDer/CircuitNetwork/web'
    f_database = dir_database + "/SBiDer.db"
    conn, cur = db_open(f_database)

    # Build path
    operon_path = build_sbider_network(dir_database, user_input, False)

    # Get operon path with names
    lst_opeNames = []
    for operon in operon_path[1:]:
        i = operon[0]
        name = i + " " + db_get_operon_name_from_id(cur, str(operon[0]))
        lst_opeNames.append(name)
    print(lst_opeNames)
    print()

# Make tester
test_file = open('/Users/Kwat/BInf/SBiDer/CircuitNetwork/web/sbider_db_all_io.txt')
tests = []
for l in test_file:
    tests.append(l[0:-1])
tests[-1] = tests[-1]+'db'

# Test
for t in tests:
    print(t)
    test(t)

laci and iptg = gfp
['1-1 pLuxI_pLacO--->gfp', '15-1 pLac--->gfp', '18-1 pLac--->gfp', '46-2 pLac--->gfp', '50-1 pLac_pBAD--->gfp', '57-1 pLac_pBAD--->LuxR_luxI']

ohhl and luxr = gfp
['1-1 pLuxI_pLacO--->gfp']

arac and lara = yfp
['2-1 pBAD--->yfp', '5-1 pBAD_pTet--->yfp', '6-1 pBAD_pLasI--->yfp']

tetr and atc = yfp
['3-1 pTet--->yfp', '5-1 pBAD_pTet--->yfp', '7-1 pTet_pLasI--->yfp', '71-2 pTet--->yfp']

lasr and pai-1 = yfp
['4-1 pLasI--->yfp', '6-1 pBAD_pLasI--->yfp', '7-1 pTet_pLasI--->yfp']

arac and lara = yfp
['2-1 pBAD--->yfp', '5-1 pBAD_pTet--->yfp', '6-1 pBAD_pLasI--->yfp']

tetr and atc = ci
['10-1 pTet_pLas--->cI', '8-1 pBAD_pTet--->cI']

arac and lara = ci
['57-1 pLac_pBAD--->LuxR_luxI', '79-1 pBAD--->cI', '8-1 pBAD_pTet--->cI', '9-1 pBAD_pLas--->cI']

lasr and pai-1 = ci
['10-1 pTet_pLas--->cI', '9-1 pBAD_pLas--->cI']

rhlr and pai-2 = ci


TypeError: 'NoneType' object is not subscriptable